In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

import pickle
import numpy as np
import os


In [2]:
from google.colab import files
uploaded = files.upload()


Saving pap.txt to pap.txt


In [3]:

file = open("pap.txt", "r", encoding = "utf8")

lines = []
for i in file:
    lines.append(i)

data = ""
for i in lines:
  data = ' '. join(lines) 

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Historical sketches of Old Charing. The hospital and chapel of Saint Mary Roncevall. Eleanor of Castile, Queen of England, and the monuments erected in her memory., by James Galloway This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at w'

In [4]:
len(data)

196100

In [5]:
wrds = data.split()
len(wrds)

32896

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

sequence_data[:15]

[1, 36, 33, 311, 2, 514, 654, 2, 168, 40, 1, 31, 3, 46, 2]

In [7]:
len(sequence_data)

33270

In [8]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5160


In [9]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
  
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  33267


array([[  1,  36,  33, 311],
       [ 36,  33, 311,   2],
       [ 33, 311,   2, 514],
       [311,   2, 514, 654],
       [  2, 514, 654,   2],
       [514, 654,   2, 168],
       [654,   2, 168,  40],
       [  2, 168,  40,   1],
       [168,  40,   1,  31],
       [ 40,   1,  31,   3]])

In [10]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [11]:
y = to_categorical(y, num_classes=vocab_size)
y[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             51600     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 5160)              5165160   
                                                                 
Total params: 18,265,760
Trainable params: 18,265,760
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=256, callbacks=[checkpoint])

Epoch 1/50
520/520 [==============================] - ETA: 0s - loss: 6.6703 - accuracy: 0.1020
Epoch 1: loss improved from inf to 6.67030, saving model to next_words.h5
520/520 [==============================] - 262s 495ms/step - loss: 6.6703 - accuracy: 0.1020
Epoch 2/50
 38/520 [=>............................] - ETA: 3:54 - loss: 6.1233 - accuracy: 0.1398

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue